In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [4]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic']

In [5]:
df= sns.load_dataset('planets')
df.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


In [7]:
df.orbital_period.isna().sum()
orbital_period_data = df.orbital_period.dropna()

In [8]:
true_mean= df.orbital_period.mean() 
true_std= df.orbital_period.std()
print(true_mean, ',', true_std)

2002.9175960947582 , 26014.728304062508


- Create 95% and 99% CI for orbital_period and check how many of these actually contain the true population mean. Give your reasoning for the sample size you pick.
- Do the above simulation for a) population std dev is known b) population std is unknown i.e calculate from the sample

In [9]:
df.orbital_period.skew()

np.float64(24.273066584470723)

In [13]:
# Function to calculate confidence intervals when population std dev is known
def calculate_confidence_interval_known_std(sample, population_std, confidence_level):
    sample_mean = np.mean(sample)
    sample_size = len(sample)
    
    # z-scores for 95% and 99% confidence intervals
    z_scores = {0.95: 1.96, 0.99: 2.576}
    c = z_scores[confidence_level]
    
    # Standard Error
    se = population_std / np.sqrt(sample_size)
    
    # Margin of Error
    me = c * se
    
    return (sample_mean - me, sample_mean + me)

# Function to calculate confidence intervals when population std dev is unknown
def calculate_confidence_interval_unknown_std(sample, confidence_level):
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)
    sample_size = len(sample)
    
    # z-scores for 95% and 99% confidence intervals
    z_scores = {0.95: 1.96, 0.99: 2.576}
    c = z_scores[confidence_level]
    
    # Standard Error
    se = sample_std / np.sqrt(sample_size)
    
    # Margin of Error
    me = c * se
    
    return (sample_mean - me, sample_mean + me)


In [15]:
# Simulation parameters
sample_size = 100
num_samples = 10000
confidence_levels = [0.95, 0.99]

# Lists to store the results
contains_true_mean_known = {0.95: 0, 0.99: 0}
contains_true_mean_unknown = {0.95: 0, 0.99: 0}

# Run the simulations
for _ in range(num_samples):
    sample = np.random.choice(orbital_period_data, size=sample_size, replace=True)
    
    for confidence_level in confidence_levels:
        ci_known = calculate_confidence_interval_known_std(sample, true_std, confidence_level)
        if ci_known[0] <= true_mean <= ci_known[1]:
            contains_true_mean_known[confidence_level] += 1
        
        ci_unknown = calculate_confidence_interval_unknown_std(sample, confidence_level)
        if ci_unknown[0] <= true_mean <= ci_unknown[1]:
            contains_true_mean_unknown[confidence_level] += 1

# Print the results
for confidence_level in confidence_levels:
    print(f"\nConfidence Level: {confidence_level*100}%")
    print(f"Known Std Dev - Intervals containing true mean: {contains_true_mean_known[confidence_level]}/{num_samples}")
    print(f"Unknown Std Dev - Intervals containing true mean: {contains_true_mean_unknown[confidence_level]}/{num_samples}")



Confidence Level: 95.0%
Known Std Dev - Intervals containing true mean: 9036/10000
Unknown Std Dev - Intervals containing true mean: 3700/10000

Confidence Level: 99.0%
Known Std Dev - Intervals containing true mean: 9763/10000
Unknown Std Dev - Intervals containing true mean: 4229/10000
